# Nested logit

In [ ]:
import numpy as np
import os
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from IPython.display import display, HTML

In [ ]:
thePath = os.getcwd().split("veteran_students_mec_optim\\Pauline")[0]
travelmode = pd.read_csv(thePath + 'data_mec_optim\\demand_travelmode\\travelmodedata.csv', sep=',')

In [ ]:
travelmode.head()

In [ ]:
lb = LabelEncoder() 
travelmode['choice'] = lb.fit_transform(travelmode['choice'])

In [ ]:
travelmode.head()

In [ ]:
nobs = travelmode.shape[0]
ncols = travelmode.shape[1]
nbchoices = 4
ninds = int(nobs/nbchoices)

In [ ]:
muhat_i_y = travelmode['choice'].values.reshape(ninds,4).T
muhat_iy = muhat_i_y.flatten()

In [ ]:
s_y = travelmode.groupby(['mode']).mean()['choice'].to_frame().sort_index()
s_y

In [ ]:
Ulogit = np.log(s_y/s_y.loc[['car']].values[0])
Ulogit

In [ ]:
l = [0.5, 0.5]
nest = Ulogit.index.isin(['car'])
Unocar = s_y[~nest].apply(np.log).multiply(l[0]).add((1-l[0])*np.sum(s_y[~nest]).apply(np.log))
display(Unocar)
Ucar = s_y[nest].apply(np.log).multiply(l[1]).add((1-l[1])*np.sum(s_y[nest]).apply(np.log))
display(Ucar)

Systematic utilities (nested logit):

In [ ]:
Unested = pd.concat([Unocar,Ucar]).add(-Ucar.values[0]).sort_index()
Unested

Choice probabilities within nocar nest (predicted vs observed):

In [ ]:
display(Unested[~nest].multiply(1/l[0]).apply(np.exp).divide(np.sum(Unested[~nest].multiply(1/l[0]).apply(np.exp))))
display(s_y[~nest]/np.sum(s_y[~nest]))

Choice probabilities of car nest (predicted vs observed):

In [ ]:
display(1/(np.sum(Unested[~nest].multiply(1/l[0]).apply(np.exp))**(l[0])+1))
display(s_y.loc[['car']].values[0])